## NBA Analytics and Betting Value Analysis Notebook

In [115]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For data analysis
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns


In [116]:
import json
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime

class NBAOddsAndLineupsScraper:
    def __init__(self):
        self.session = requests.Session()
        self.setup_headers()
    
    def setup_headers(self):
        """Setup common headers for requests"""
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'max-age=0',
            'priority': 'u=0, i',
            'referer': 'https://www.rotowire.com/',
            'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        }

    # --------- RAW WIDE ODDS (no aggregation) ---------------------------------
    def get_player_props_odds_wide_raw(self, book='mgm'):
        """
        Return the raw 'wide' odds table by scraping Rotowire's player-props page.
        This preserves columns like:
          name, team, opp, <book>_pts, <book>_ptsUnder, <book>_ptsOver, ...
        Works across many books present in the page JSON blocks.
        """
        url = f"https://www.rotowire.com/betting/nba/player-props.php?book={book}"
        try:
            r = self.session.get(url, headers=self.headers)
            r.raise_for_status()
        except Exception as e:
            print(f"Failed to GET odds page: {e}")
            return pd.DataFrame()

        # Extract ALL JSON lists assigned to "data:" in the page
        matches = re.findall(r"data:\s*(\[\{.*?\}\])", r.text, flags=re.DOTALL)
        frames = []
        for m in matches:
            try:
                rows = json.loads(m)
                if isinstance(rows, list) and rows:
                    frames.append(pd.DataFrame(rows))
            except Exception:
                continue

        if not frames:
            print("No odds JSON blocks found.")
            return pd.DataFrame()

        df = pd.concat(frames, ignore_index=True)
        # keep the most useful id/basic columns if present
        base_cols = [c for c in ["name","gameID","playerID","firstName","lastName","team","opp","logo","playerLink"] if c in df.columns]
        other_cols = [c for c in df.columns if c not in base_cols]
        df = df[base_cols + other_cols]
        # normalize team/opponent field names
        if "opp" in df.columns and "opponent" not in df.columns:
            df = df.rename(columns={"opp": "opponent"})
        # add as-of date and (best-guess) game_date if not present
        df["asof_date"] = datetime.utcnow().strftime("%Y-%m-%d")
        if "game_date" not in df.columns:
            df["game_date"] = df["asof_date"]
        print(f"Successfully fetched RAW odds rows: {len(df)} | columns: {len(df.columns)}")
        return df

    # --------- Legacy aggregated method (kept in case you still call it) ------
    def get_player_props_odds(self, book='mgm'):
        """
        Old helper that aggregated rows by 'name'.
        Prefer get_player_props_odds_wide_raw() for modeling/joins.
        """
        wide = self.get_player_props_odds_wide_raw(book=book)
        if wide.empty:
            return None
        aggregated_df = wide.groupby('name', as_index=False, sort=False).agg(
            lambda x: ', '.join(pd.Series(x).dropna().astype(str).unique())
        )
        aggregated_df = aggregated_df.dropna(axis=1, how='all')
        print(f"Successfully aggregated odds for {len(aggregated_df)} players")
        return aggregated_df

    # --------- Lineups scraping (unchanged logic, made a bit sturdier) --------
    def get_expected_lineups(self):
        """Get expected lineups from Rotowire"""
        url = "https://www.rotowire.com/basketball/nba-lineups.php"
        try:
            r = self.session.get(url, headers=self.headers)
            r.raise_for_status()
        except Exception as e:
            print(f"Failed to retrieve lineup page: {e}")
            return None

        soup = BeautifulSoup(r.content, 'html.parser')
        game_containers = soup.find_all('div', class_='lineup__main')
        if not game_containers:
            print("No lineup data found")
            return None

        lineups_data = []
        for game in game_containers:
            game_info = self._parse_game_info(game)
            if game_info:
                lineups_data.append(game_info)

        print(f"Successfully fetched lineups for {len(lineups_data)} games")
        return lineups_data

    def _parse_game_info(self, game_container):
        """Parse individual game information and lineups"""
        try:
            game_data = {}
            header = game_container.find('div', class_='lineup__hdr')
            if header:
                teams = header.find_all('div', class_='lineup__team')
                if len(teams) >= 2:
                    game_data['away_team'] = teams[0].get_text(strip=True)
                    game_data['home_team'] = teams[1].get_text(strip=True)
            time_info = header.find('div', class_='lineup__time') if header else None
            if time_info:
                game_data['game_time'] = time_info.get_text(strip=True)
            lineup_containers = game_container.find_all('div', class_='lineup__box')
            if len(lineup_containers) >= 2:
                game_data['away_starters'] = self._parse_team_lineup(lineup_containers[0])
                game_data['home_starters'] = self._parse_team_lineup(lineup_containers[1])
            return game_data
        except Exception as e:
            print(f"Error parsing game info: {e}")
            return None

    def _parse_team_lineup(self, team_container):
        """Parse individual team lineup"""
        starters = []
        try:
            starters_section = team_container.find('div', class_='lineup__list')
            if starters_section:
                player_elements = starters_section.find_all('div', class_='lineup__player')
                for player_elem in player_elements:
                    player_info = self._parse_player_info(player_elem)
                    if player_info:
                        starters.append(player_info)
            return starters
        except Exception as e:
            print(f"Error parsing team lineup: {e}")
            return []

    def _parse_player_info(self, player_elem):
        """Parse individual player information"""
        try:
            player_data = {}
            name_elem = player_elem.find('a', class_='lineup__player-link')
            if name_elem:
                player_data['name'] = name_elem.get_text(strip=True)
                player_data['player_link'] = name_elem.get('href', '')
            pos_elem = player_elem.find('span', class_='lineup__pos')
            if pos_elem:
                player_data['position'] = pos_elem.get_text(strip=True)
            injury_elem = player_elem.find('span', class_='lineup__inj')
            player_data['injury_status'] = injury_elem.get_text(strip=True) if injury_elem else 'Active'
            confirmed_elem = player_elem.find('span', class_='lineup__confirm')
            player_data['confirmed_starter'] = confirmed_elem is not None
            return player_data
        except Exception as e:
            print(f"Error parsing player info: {e}")
            return None

    def get_comprehensive_data(self):
        """Get both odds and lineups data"""
        print("Fetching NBA betting data and lineups...")
        odds_data = self.get_player_props_odds_wide_raw()  # <-- use RAW wide
        lineups_data = self.get_expected_lineups()
        combined_data = {
            'odds': odds_data,
            'lineups': lineups_data,
            'last_updated': datetime.now().isoformat()
        }
        return combined_data
    
    def save_to_excel(self, data, filename=None):
        """Save the scraped data to Excel files"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f'nba_betting_data_{timestamp}.xlsx'
        try:
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                if isinstance(data.get('odds'), pd.DataFrame) and not data['odds'].empty:
                    data['odds'].to_excel(writer, sheet_name='Player_Odds', index=False)
                if data.get('lineups') is not None:
                    lineups_list = []
                    for game in data['lineups']:
                        for starter_type in ['away_starters', 'home_starters']:
                            team = game.get('away_team' if starter_type == 'away_starters' else 'home_team', 'Unknown')
                            starters = game.get(starter_type, [])
                            for starter in starters:
                                lineups_list.append({
                                    'game_time': game.get('game_time', ''),
                                    'team': team,
                                    'player_name': starter.get('name', ''),
                                    'position': starter.get('position', ''),
                                    'injury_status': starter.get('injury_status', ''),
                                    'confirmed_starter': starter.get('confirmed_starter', False),
                                    'player_link': starter.get('player_link', '')
                                })
                    if lineups_list:
                        lineups_df = pd.DataFrame(lineups_list)
                        lineups_df.to_excel(writer, sheet_name='Expected_Lineups', index=False)
                metadata = pd.DataFrame([{
                    'last_updated': data.get('last_updated', ''),
                    'total_games': len(data.get('lineups', [])) if isinstance(data.get('lineups'), list) else 0,
                    'total_players_odds': len(data.get('odds', [])) if isinstance(data.get('odds'), pd.DataFrame) else 0
                }])
                metadata.to_excel(writer, sheet_name='Metadata', index=False)
            print(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            print(f"Error saving to Excel: {e}")
            return False

# Usage example and integration with your existing analytics
def integrate_with_analytics():
    """Integrate the scraper with your existing analytics"""
    scraper = NBAOddsAndLineupsScraper()
    nba_data = scraper.get_comprehensive_data()
    scraper.save_to_excel(nba_data)
    processed_data = process_for_analytics(nba_data)
    return processed_data

def process_for_analytics(nba_data):
    """Process the scraped data for use in analytics"""
    processed = {}
    # Odds data → extract basic lines for PTS/REB/AST if present
    if isinstance(nba_data.get('odds'), pd.DataFrame) and not nba_data['odds'].empty:
        odds_df = nba_data['odds']
        def pick_line(row, market):
            # Look for any <book>_<marketLower> columns (line, Under, Over)
            m = market.lower()
            line = None
            over = None
            under = None
            for col in row.index:
                c = col.lower()
                if c.endswith(f"_{m}"):
                    line = row[col]
                elif c.endswith(f"_{m}over"):
                    over = row[col]
                elif c.endswith(f"_{m}under"):
                    under = row[col]
            try:
                line = float(line) if line is not None and str(line).strip() not in ("", "None", "nan") else None
            except Exception:
                line = None
            return line, over, under

        betting_lines = []
        for _, r in odds_df.iterrows():
            player_name = r.get('name', '')
            for mk in ["pts", "reb", "ast"]:
                line, over, under = pick_line(r, mk)
                if line is not None:
                    betting_lines.append({
                        "player": player_name,
                        "stat": {"pts":"points","reb":"rebounds","ast":"assists"}[mk],
                        "line": line,
                        "over_odds": over,
                        "under_odds": under
                    })
        processed['betting_lines'] = pd.DataFrame(betting_lines)

    # Lineups
    if nba_data.get('lineups') is not None:
        lineups = nba_data['lineups']
        team_players = {}
        for game in lineups:
            away_team = game.get('away_team')
            if away_team and away_team not in team_players:
                team_players[away_team] = []
            for starter in game.get('away_starters', []):
                if away_team and starter.get('name'):
                    team_players[away_team].append({
                        'name': starter['name'],
                        'position': starter.get('position', ''),
                        'status': starter.get('injury_status', 'Active'),
                        'confirmed': starter.get('confirmed_starter', False)
                    })
            home_team = game.get('home_team')
            if home_team and home_team not in team_players:
                team_players[home_team] = []
            for starter in game.get('home_starters', []):
                if home_team and starter.get('name'):
                    team_players[home_team].append({
                        'name': starter['name'],
                        'position': starter.get('position', ''),
                        'status': starter.get('injury_status', 'Active'),
                        'confirmed': starter.get('confirmed_starter', False)
                    })
        processed['team_lineups'] = team_players
        processed['games_today'] = lineups
    return processed

def extract_betting_line(player_row, stat_type):
    """Extract betting line for specific stat type (legacy helper)"""
    line_col = over_odds_col = under_odds_col = None
    for col in player_row.index:
        col_lower = col.lower()
        if stat_type in col_lower and 'line' in col_lower:
            line_col = col
        elif stat_type in col_lower and 'over' in col_lower and 'odds' in col_lower:
            over_odds_col = col
        elif stat_type in col_lower and 'under' in col_lower and 'odds' in col_lower:
            under_odds_col = col
    line_value = player_row.get(line_col) if line_col else None
    if line_value and str(line_value).replace('.', '').isdigit():
        return {
            'line': float(line_value),
            'over_odds': player_row.get(over_odds_col) if over_odds_col else None,
            'under_odds': player_row.get(under_odds_col) if under_odds_col else None
        }
    return None

# Main execution
if __name__ == "__main__":
    data = integrate_with_analytics()
    print("\n" + "="*50)
    print("NBA BETTING DATA SUMMARY")
    print("="*50)
    if data.get('betting_lines') is not None:
        print(f"\nBetting Lines: {len(data['betting_lines'])} player-stat combinations")
        print(data['betting_lines'].head(10))
    if data.get('team_lineups'):
        print(f"\nTeams with Lineups: {len(data['team_lineups'])}")
        for team, players in list(data['team_lineups'].items())[:3]:
            print(f"{team}: {len(players)} players")
            for player in players[:3]:
                print(f"  - {player['name']} ({player['position']}) - {player['status']}")
    if data.get('games_today'):
        print(f"\nGames Today: {len(data['games_today'])}")
        for game in data['games_today'][:3]:
            print(f"{game.get('away_team', 'TBD')} @ {game.get('home_team', 'TBD')} - {game.get('game_time', 'Time TBD')}")


Fetching NBA betting data and lineups...
Successfully fetched RAW odds rows: 1695 | columns: 263
Successfully fetched lineups for 0 games
Data successfully saved to nba_betting_data_20251102_204000.xlsx

NBA BETTING DATA SUMMARY

Betting Lines: 0 player-stat combinations
Empty DataFrame
Columns: []
Index: []


In [117]:
def get_daily_matchups(date=None):
    """Get NBA games for a specific date"""
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
    # Placeholder demo; replace with a real schedule API if desired
    sample_matchups = [
        {'home_team': 'GSW', 'away_team': 'LAL', 'time': '7:30 PM ET'},
        {'home_team': 'BOS', 'away_team': 'MIA', 'time': '8:00 PM ET'},
        {'home_team': 'DEN', 'away_team': 'DAL', 'time': '9:00 PM ET'},
    ]
    return sample_matchups

def calculate_player_correlations(player_a_logs, player_b_logs):
    """Calculate correlation between two players' performances"""
    merged = pd.merge(player_a_logs, player_b_logs, on='GAME_DATE', suffixes=('_a', '_b'))
    correlations = {}
    for stat in ['PTS', 'REB', 'AST']:
        if f'{stat}_a' in merged.columns and f'{stat}_b' in merged.columns:
            corr = merged[f'{stat}_a'].corr(merged[f'{stat}_b'])
            correlations[stat] = corr
    return correlations

# Export results to Excel
def export_analysis(results, filename='nba_betting_analysis.xlsx'):
    """Export analysis results to Excel"""
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        if 'value_bets' in results:
            pd.DataFrame(results['value_bets']).to_excel(writer, sheet_name='Value_Bets', index=False)
        if 'predictions' in results:
            predictions_df = pd.DataFrame.from_dict(results['predictions'], orient='index')
            predictions_df.to_excel(writer, sheet_name='Player_Predictions')
    print(f"Analysis exported to {filename}")


## NBA

In [118]:
import requests
import pandas as pd
import time
import random

url = "https://stats.nba.com/stats/leaguedashplayerstats"

base_params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "ISTRound": "",
    "LastNGames": "0",
    "LeagueID": "00",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": ""
}

headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Origin": "https://www.nba.com",
    "Referer": "https://www.nba.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36",
    "x-nba-stats-origin": "stats",
    "x-nba-stats-token": "true"
}

seasons = ["2023-24", "2024-25"]

def fetch_season_data(season, retries=3):
    """Fetch one season’s player stats, retrying if timeout or network error."""
    params = base_params.copy()
    params["Season"] = season

    for attempt in range(1, retries + 1):
        try:
            print(f"→ Attempt {attempt} fetching {season} data...")
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.Timeout:
            print(f"⚠️ Timeout on attempt {attempt}/{retries} for {season}. Retrying...")
            time.sleep(3 * attempt)
        except requests.exceptions.RequestException as e:
            print(f"❌ Error on attempt {attempt}/{retries}: {e}")
            time.sleep(3 * attempt)
    raise RuntimeError(f"Failed to fetch {season} data after {retries} attempts.")

# Main loop
for season in seasons:
    print(f"\n🏀 Fetching NBA stats for {season}...")
    data = fetch_season_data(season)

    headers_list = data["resultSets"][0]["headers"]
    rows = data["resultSets"][0]["rowSet"]

    df = pd.DataFrame(rows, columns=headers_list)
    filename = f"nba_player_stats_{season.replace('-', '_')}.csv"
    df.to_csv(filename, index=False)

    print(f"✅ {season}: saved {len(df)} player records to '{filename}'")

    # Wait 3–6 seconds before next season to avoid throttling
    time.sleep(random.uniform(3, 6))

print("\n🎉 Done! Both 2023-24 and 2024-25 seasons downloaded.")



🏀 Fetching NBA stats for 2023-24...
→ Attempt 1 fetching 2023-24 data...
✅ 2023-24: saved 572 player records to 'nba_player_stats_2023_24.csv'

🏀 Fetching NBA stats for 2024-25...
→ Attempt 1 fetching 2024-25 data...
✅ 2024-25: saved 569 player records to 'nba_player_stats_2024_25.csv'

🎉 Done! Both 2023-24 and 2024-25 seasons downloaded.


## GAME LOGS

In [119]:
import requests
import pandas as pd
import time

def get_box_scores(season, season_type="Regular Season"):
    url = "https://stats.nba.com/stats/leaguegamelog"
    params = {
        "Counter": 1000,
        "DateFrom": "",
        "DateTo": "",
        "Direction": "DESC",
        "ISTRound": "",
        "LeagueID": "00",
        "PlayerOrTeam": "P",
        "Season": season,
        "SeasonType": season_type,
        "Sorter": "DATE"
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
        "Referer": "https://www.nba.com/",
        "Origin": "https://www.nba.com",
        "Accept": "application/json, text/plain, */*"
    }

    response = requests.get(url, params=params, headers=headers, timeout=30)
    response.raise_for_status()

    data = response.json()["resultSets"][0]
    df = pd.DataFrame(data["rowSet"], columns=data["headers"])
    return df

# Get all three seasons
seasons = ["2023-24", "2024-25", "2025-26"]
for season in seasons:
    print(f"Fetching {season}...")
    df = get_box_scores(season)
    df.to_csv(f"nba_boxscores_{season}.csv", index=False)
    print(f"✅ Saved {len(df)} records for {season}")
    time.sleep(2)  # polite delay


Fetching 2023-24...
✅ Saved 26401 records for 2023-24
Fetching 2024-25...
✅ Saved 26306 records for 2024-25
Fetching 2025-26...
✅ Saved 1933 records for 2025-26


In [120]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import requests
import io
import unicodedata

# ---- Keep/Map settings -------------------------------------------------------

ADV_COLS_KEEP = [
    "Player", "Pos", "Age", "Tm", "G", "MP",
    "PER", "TS%", "3PAr", "FTr",
    "ORB%", "DRB%", "TRB%",
    "AST%", "STL%", "BLK%",
    "TOV%", "USG%",
    "ORtg", "DRtg",
    "OWS", "DWS", "WS", "WS/48",
    "OBPM", "DBPM", "BPM", "VORP"
]

# Basketball-Reference -> NBA/your dataset codes
TEAM_ABBR_MAP = {
    "BRK": "BKN",
    "PHO": "PHX",
    "CHO": "CHA",
    "UTH": "UTA",   # rare alias safety
    "NJN": "BKN",   # historical
    "SEA": "OKC",   # historical
    "VAN": "MEM",   # historical
}

# ---- Helpers -----------------------------------------------------------------

def normalize_name(s):
    """Normalize player names for consistent joining (lowercase, no accents/punct)."""
    if pd.isna(s):
        return s
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    for ch in [".", "'", "`", "’", "“", "”", ","]:
        s = s.replace(ch, "")
    s = " ".join(s.split())
    return s

# ---- Fetch advanced table from Basketball-Reference --------------------------

def fetch_advanced_table(season=2026):
    """
    Fetch and clean Basketball-Reference advanced stats table for a given season.
    Example: season=2025 -> https://www.basketball-reference.com/leagues/NBA_2025_advanced.html
    """
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    }
    resp = requests.get(url, headers=headers, timeout=30)
    resp.raise_for_status()

    tables = pd.read_html(io.StringIO(resp.text), header=0)
    if not tables:
        raise RuntimeError("No tables found on Basketball-Reference page.")

    df = tables[0].copy()

    # Remove duplicate header rows
    if "Rk" in df.columns:
        df = df[df["Rk"] != "Rk"].copy()
        df.drop(columns=["Rk"], inplace=True, errors="ignore")

    # Normalize column names (strip and upper-case for easy access)
    df.columns = [c.strip() for c in df.columns]

    # Basketball Reference sometimes labels the team column differently — make sure it exists
    team_col = None
    for c in df.columns:
        if c.lower() in ["tm", "team", "team_name"]:
            team_col = c
            break
    if not team_col:
        raise KeyError(f"Could not find a team column in advanced table. Found: {df.columns.tolist()}")
    df.rename(columns={team_col: "Tm"}, inplace=True)

    # Keep relevant columns if present
    keep = [c for c in ADV_COLS_KEEP if c in df.columns]
    df = df[keep].copy()

    # Convert numeric columns
    non_numeric = {"Player", "Pos", "Tm"}
    for c in [c for c in df.columns if c not in non_numeric]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Map team abbreviations to match your dataset
    df["Tm"] = df["Tm"].replace(TEAM_ABBR_MAP)

    # Add join keys
    df["player_key"] = df["Player"].map(normalize_name)
    df["team_key"] = df["Tm"].astype(str).str.strip().str.upper()

    return df

# ---- Load your averages CSV and align columns --------------------------------

def load_averages_csv(path):
    """
    Load your NBA averages CSV (with headers like PLAYER_NAME, TEAM_ABBREVIATION).
    Renames to canonical 'Player' and 'Team' and adds join keys.
    """
    df = pd.read_csv(path)

    # Auto-map your headers to canonical names
    col_map = {}
    for c in df.columns:
        cl = c.strip().lower()
        if cl == "player_name":
            col_map[c] = "Player"
        elif cl in ("team_abbreviation", "tm", "team"):
            col_map[c] = "Team"
        # keep other columns as-is

    df = df.rename(columns=col_map)

    if "Player" not in df.columns or "Team" not in df.columns:
        raise ValueError(
            "Couldn't find columns for 'Player' and 'Team'. "
            f"Available columns: {list(df.columns)}"
        )

    # Join keys
    df["player_key"] = df["Player"].map(normalize_name)
    df["team_key"] = df["Team"].astype(str).str.strip().str.upper()

    return df

# ---- Merge logic (with TOT fallback for traded players) ----------------------

def merge_advanced_into_averages(df_avg, df_adv):
    """
    Merge advanced metrics into averages.
    1) Exact Player+Team match (ignore TOT).
    2) For remaining NaNs, fill from TOT row by Player.
    """
    adv_team = df_adv[df_adv["Tm"] != "TOT"].copy()
    adv_tot  = df_adv[df_adv["Tm"] == "TOT"].copy()

    adv_cols_to_add = [c for c in df_adv.columns if c not in {"Player", "Pos", "Age", "Tm", "player_key", "team_key"}]
    meta_cols = [c for c in ["Pos", "Age"] if c in df_adv.columns]
    join_cols_full = meta_cols + adv_cols_to_add

    merged = df_avg.merge(
        adv_team[["player_key", "team_key"] + join_cols_full],
        on=["player_key", "team_key"],
        how="left",
        suffixes=("", "_adv"),
    )

    # Determine "missing" based on a representative advanced column
    probe_col = "PER" if "PER" in merged.columns else ("WS/48" if "WS/48" in merged.columns else None)
    missing_mask = merged[probe_col].isna() if probe_col else merged.isna().any(axis=1)

    if missing_mask.any() and not adv_tot.empty:
        fallback = merged[missing_mask].merge(
            adv_tot[["player_key"] + join_cols_full],
            on="player_key",
            how="left",
            suffixes=("", "_tot"),
        )
        for col in join_cols_full:
            if col in merged.columns and col in fallback.columns:
                merged.loc[missing_mask, col] = merged.loc[missing_mask, col].fillna(fallback[col])

    return merged

# ==============================================================================
# Example usage for your two files
# ==============================================================================

# ---- 2023–24 (Basketball-Reference season code = 2024) -----------------------
df_avg_2024 = load_averages_csv("nba_player_stats_2023_24.csv")
df_adv_2024 = fetch_advanced_table(season=2024)
df_enriched_2024 = merge_advanced_into_averages(df_avg_2024, df_adv_2024)
df_enriched_2024.to_csv("nba_player_stats_2023_24_enriched.csv", index=False)
print("✅ Saved: nba_player_stats_2023_24_enriched.csv")

# ---- 2024–25 (Basketball-Reference season code = 2025) -----------------------
df_avg_2025 = load_averages_csv("nba_player_stats_2024_25.csv")
df_adv_2025 = fetch_advanced_table(season=2025)
df_enriched_2025 = merge_advanced_into_averages(df_avg_2025, df_adv_2025)
df_enriched_2025.to_csv("nba_player_stats_2024_25_enriched.csv", index=False)
print("✅ Saved: nba_player_stats_2024_25_enriched.csv")

# ---- (Optional) Combine both seasons into one file ---------------------------
df_combined = pd.concat([df_enriched_2024, df_enriched_2025], ignore_index=True)
df_combined.to_csv("nba_player_stats_2023_25_combined.csv", index=False)
print("🏀 Combined: nba_player_stats_2023_25_combined.csv")


✅ Saved: nba_player_stats_2023_24_enriched.csv
✅ Saved: nba_player_stats_2024_25_enriched.csv
🏀 Combined: nba_player_stats_2023_25_combined.csv


## Analysis

In [121]:
import pandas as pd
import numpy as np

# -----------------------------
# Input file assumptions:
# - You have player game logs with at least:
#   ['GAME_DATE', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'OPPONENT_ABBREVIATION',
#    'MIN', 'PTS', 'REB', 'AST', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'TOV', 'BLK', 'STL', 'PLUS_MINUS', 'START_POSITION']
#   Column names can be auto-mapped below if they differ slightly (e.g., 'TEAM_ID' not needed).
# -----------------------------

def standardize_logs_cols(df_logs: pd.DataFrame) -> pd.DataFrame:
    colmap = {}
    for c in df_logs.columns:
        cl = c.strip().lower()
        if cl in ["game_date", "game_date_est", "date"]:
            colmap[c] = "GAME_DATE"
        elif cl in ["player", "player_name"]:
            colmap[c] = "PLAYER_NAME"
        elif cl in ["team", "team_abbreviation", "tm"]:
            colmap[c] = "TEAM_ABBREVIATION"
        elif cl in ["opp", "opponent", "opponent_abbreviation"]:
            colmap[c] = "OPPONENT_ABBREVIATION"
        elif cl in ["min", "minutes"]:
            colmap[c] = "MIN"
    df = df_logs.rename(columns=colmap).copy()
    # types
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
    df = df.sort_values(["PLAYER_NAME", "GAME_DATE"])
    return df

def add_shooting_efficiency(df: pd.DataFrame) -> pd.DataFrame:
    # Compute TS% from game logs (per game)
    # TS% = PTS / (2*(FGA + 0.44*FTA))
    for col in ["FGA", "FTA", "PTS"]:
        if col not in df.columns:
            df[col] = 0.0
    denom = 2 * (df["FGA"].astype(float) + 0.44 * df["FTA"].astype(float))
    df["TS_game"] = np.where(denom > 0, df["PTS"].astype(float) / denom, np.nan)
    return df

def rolling_player_form(df: pd.DataFrame, windows=(3,5,10,20)) -> pd.DataFrame:
    # Rolling stats per player before each game
    df = df.sort_values(["PLAYER_NAME", "GAME_DATE"]).copy()
    group = df.groupby("PLAYER_NAME", group_keys=False)
    for w in windows:
        for stat in ["PTS", "REB", "AST", "MIN", "TS_game"]:
            if stat not in df.columns:
                df[stat] = np.nan
            col = f"{stat}_roll{w}"
            df[col] = group[stat].shift(1).rolling(w, min_periods=1).mean()
    # recent usage proxy: last-5 share of team FGA
    if {"FGA","TEAM_ABBREVIATION"}.issubset(df.columns):
        df["teamFGA_game"] = df.groupby(["TEAM_ABBREVIATION","GAME_DATE"])["FGA"].transform("sum")
        df["usage_share"] = np.where(df["teamFGA_game"]>0, df["FGA"]/df["teamFGA_game"], np.nan)
        df["usage_share_roll5"] = group["usage_share"].shift(1).rolling(5, min_periods=1).mean()
    return df

def team_daily_ratings(df: pd.DataFrame, windows=(5,10)):
    # Build team-level ORtg/DRtg/Pace rolling using box score approximations
    # Possessions ≈ FGA + 0.44*FTA - OREB + TOV (OREB optional if present)
    need_cols = ["TEAM_ABBREVIATION","OPPONENT_ABBREVIATION","GAME_DATE","PTS","FGA","FTA","TOV","OREB"]
    for c in need_cols:
        if c not in df.columns:
            df[c] = 0.0
    # aggregate team totals per game
    g = df.groupby(["GAME_DATE","TEAM_ABBREVIATION"], as_index=False).agg(
        PTS_team=("PTS","sum"), FGA=("FGA","sum"), FTA=("FTA","sum"),
        TOV=("TOV","sum"), OREB=("OREB","sum")
    )
    g["poss"] = g["FGA"] + 0.44*g["FTA"] - g["OREB"] + g["TOV"]
    # opponent join to get DRtg inputs
    opp = g.rename(columns={
        "TEAM_ABBREVIATION":"OPPONENT_ABBREVIATION",
        "PTS_team":"PTS_opp",
        "poss":"poss_opp"
    })[["GAME_DATE","OPPONENT_ABBREVIATION","PTS_opp","poss_opp"]]
    g2 = g.merge(opp, on=["GAME_DATE"], how="left")
    # approximate per-team DRtg from opponent scoring
    g2["ORtg_g"] = np.where(g2["poss"]>0, 100*g2["PTS_team"]/g2["poss"], np.nan)
    g2["DRtg_g"] = np.where(g2["poss_opp"]>0, 100*g2["PTS_opp"]/g2["poss_opp"], np.nan)
    g2["Pace_g"] = (g2["poss"] + g2["poss_opp"]) / 2.0
    g2 = g2.sort_values(["TEAM_ABBREVIATION","GAME_DATE"])
    # rolling
    out = g2.copy()
    for w in windows:
        for stat in ["ORtg_g","DRtg_g","Pace_g"]:
            out[f"{stat}_roll{w}"] = out.groupby("TEAM_ABBREVIATION")[stat].shift(1).rolling(w, min_periods=1).mean()
    return out[["GAME_DATE","TEAM_ABBREVIATION","ORtg_g_roll5","DRtg_g_roll5","Pace_g_roll5",
                "ORtg_g_roll10","DRtg_g_roll10","Pace_g_roll10"]]

def opponent_position_allowances(df: pd.DataFrame, window=10):
    # How many points/assists/rebounds a team allows per opponent position (rolling)
    if "START_POSITION" not in df.columns:
        df["START_POSITION"] = np.nan  # if not available, this will be sparse
    base = df.groupby(["GAME_DATE","OPPONENT_ABBREVIATION","START_POSITION"], as_index=False)\
             .agg(PTS_allowed=("PTS","sum"), AST_allowed=("AST","sum"), REB_allowed=("REB","sum"))
    base = base.sort_values(["OPPONENT_ABBREVIATION","START_POSITION","GAME_DATE"])
    for w in [window]:
        for stat in ["PTS_allowed","AST_allowed","REB_allowed"]:
            base[f"{stat}_roll{w}"] = base.groupby(["OPPONENT_ABBREVIATION","START_POSITION"])[stat]\
                                            .shift(1).rolling(w, min_periods=3).mean()
    # pivot to wide per opponent (columns per position)
    wide = base.pivot_table(index=["GAME_DATE","OPPONENT_ABBREVIATION"],
                            columns="START_POSITION",
                            values=[f"PTS_allowed_roll{window}",f"AST_allowed_roll{window}",f"REB_allowed_roll{window}"])
    wide.columns = [f"{a}_{b}" for a,b in wide.columns.to_flat_index()]
    wide = wide.reset_index()
    return wide

def assemble_player_game_features(df_logs: pd.DataFrame, df_enriched_season: pd.DataFrame) -> pd.DataFrame:
    df = standardize_logs_cols(df_logs)
    df = add_shooting_efficiency(df)
    df = rolling_player_form(df)

    # Team rolling ratings
    tr = team_daily_ratings(df)
    df = df.merge(tr, on=["GAME_DATE","TEAM_ABBREVIATION"], how="left")

    # Opponent allowances by position
    oppw = opponent_position_allowances(df)
    df = df.merge(oppw, left_on=["GAME_DATE","OPPONENT_ABBREVIATION"], right_on=["GAME_DATE","OPPONENT_ABBREVIATION"], how="left")

    # Merge season-enriched averages (PER/TS%/USG%/ORtg/DRtg etc.)
    tmp = df_enriched_season.copy()
    # normalize keys like before
    def _norm(s):
        import unicodedata
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s)
        s = "".join(ch for ch in s if not unicodedata.combining(ch))
        for ch in [".","'","`","’","“","”",","]:
            s = s.replace(ch,"")
        return " ".join(s.split())
    df["player_key"] = df["PLAYER_NAME"].map(_norm)
    df["team_key"] = df["TEAM_ABBREVIATION"].str.upper()

    tmp["player_key"] = tmp["Player"].map(_norm)
    tmp["team_key"] = tmp["Team"].astype(str).str.upper()

    keep_adv = [c for c in ["PER","TS%","USG%","ORtg","DRtg","WS/48","BPM","VORP","Pos","Age"] if c in tmp.columns]
    df = df.merge(tmp[["player_key","team_key"] + keep_adv], on=["player_key","team_key"], how="left")

    # simple situational flags
    if "MATCHUP" in df.columns:
        df["HOME"] = df["MATCHUP"].str.contains(" vs. ", regex=False).astype(int)
    else:
        df["HOME"] = np.nan  # placeholder

    # Days rest
    df["prev_date"] = df.groupby("PLAYER_NAME")["GAME_DATE"].shift(1)
    df["days_rest"] = (df["GAME_DATE"] - df["prev_date"]).dt.days

    # Targets: next-game points, rebounds, assists
    df = df.sort_values(["PLAYER_NAME","GAME_DATE"])
    for target, src in [("PTS_next","PTS"), ("REB_next","REB"), ("AST_next","AST")]:
        if src not in df.columns:
            df[src] = np.nan
        df[target] = df.groupby("PLAYER_NAME")[src].shift(-1)

    return df


In [122]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

# --- Load your logs and enriched season files (paths must match Cell 4 / 5 outputs) ---
logs_2324 = pd.read_csv("nba_boxscores_2023-24.csv")        # adjust if your path differs
logs_2425 = pd.read_csv("nba_boxscores_2024-25.csv")
enriched_2324 = pd.read_csv("nba_player_stats_2023_24_enriched.csv")
enriched_2425 = pd.read_csv("nba_player_stats_2024_25_enriched.csv")

# --- Build feature tables per season and concatenate ---
feat_2324 = assemble_player_game_features(logs_2324, enriched_2324)
feat_2425 = assemble_player_game_features(logs_2425, enriched_2425)
features_all = pd.concat([feat_2324, feat_2425], ignore_index=True)

# --- Base feature pool (we'll filter by existence) ---
BASE_FEATURES = [
    # rolling form (generic)
    "MIN_roll5","MIN_roll10","TS_game_roll5","TS_game_roll10","usage_share_roll5",
    # team context
    "ORtg_g_roll5","DRtg_g_roll5","Pace_g_roll5",
    # advanced season context
    "PER","TS%","USG%","ORtg","DRtg","WS/48","BPM","VORP",
    # rest / home
    "days_rest","HOME"
]

# Stat-specific rolling features to add per target
STAT_ROLLING = {
    "PTS": ["PTS_roll5","PTS_roll10"],
    "REB": ["REB_roll5","REB_roll10"],
    "AST": ["AST_roll5","AST_roll10"],
}

# Targets
TARGETS = {
    "PTS": "PTS_next",
    "REB": "REB_next",
    "AST": "AST_next",
}

# Containers for later cells
models = {}                 # e.g., models["PTS"] = fitted RF
feature_cols_by_stat = {}   # e.g., feature_cols_by_stat["PTS"] = [...]
cv_scores = {}              # MAE per stat

# Time series CV
tscv = TimeSeriesSplit(n_splits=5)

# Train one model per stat
for stat, target_col in TARGETS.items():
    # Build the candidate features for this stat
    cand_feats = BASE_FEATURES + STAT_ROLLING[stat]
    # Keep only columns that exist
    feat_cols = [c for c in cand_feats if c in features_all.columns]
    feature_cols_by_stat[stat] = feat_cols

    # Drop rows with missing features/target
    data = features_all.dropna(subset=feat_cols + [target_col]).copy()
    if data.empty:
        print(f"⚠️ No training rows available for {stat} (missing features/target). Skipping.")
        continue

    # Sort by time to avoid leakage
    data_sorted = data.sort_values("GAME_DATE")
    X = data_sorted[feat_cols]
    y = data_sorted[target_col]

    # CV loop
    maes = []
    for train_idx, test_idx in tscv.split(X):
        Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
        ytr, yte = y.iloc[train_idx], y.iloc[test_idx]
        model = RandomForestRegressor(
            n_estimators=400,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        model.fit(Xtr, ytr)
        pred = model.predict(Xte)
        maes.append(mean_absolute_error(yte, pred))

    cv_scores[stat] = (float(np.mean(maes)), float(np.std(maes)))
    print(f"Baseline Player {stat} MAE (TimeSeries CV): {np.mean(maes):.2f} ± {np.std(maes):.2f}")

    # Fit final model on all data for this stat
    final_model = RandomForestRegressor(
        n_estimators=400,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    final_model.fit(X, y)
    models[stat] = final_model

# For convenience (optional): keep last-trained model/feature_cols for Cell 8 fallback
# (Cell 8 below will use the per-stat dict to show importances for each target.)
if "PTS" in models:
    model = models["PTS"]
    feature_cols = feature_cols_by_stat["PTS".]


Baseline Player PTS MAE (TimeSeries CV): 4.86 ± 0.06
Baseline Player REB MAE (TimeSeries CV): 2.10 ± 0.03
Baseline Player AST MAE (TimeSeries CV): 1.47 ± 0.03


In [123]:
import pandas as pd

if not models:
    raise RuntimeError("No models trained in Cell 7. Ensure features/targets exist and rerun Cell 7.")

all_imps = {}
for stat, mdl in models.items():
    feat_cols = feature_cols_by_stat.get(stat, [])
    if hasattr(mdl, "feature_importances_"):
        imp_series = pd.Series(mdl.feature_importances_, index=feat_cols).sort_values(ascending=False)
        all_imps[stat] = imp_series
        print(f"\nTop 15 importances — {stat}:")
        display(imp_series.head(15))
    else:
        print(f"\nModel for {stat} has no feature_importances_ attribute.")

# Keep the most recently shown importances in 'imp' for backward compatibility
if "PTS" in all_imps:
    imp = all_imps["PTS"]



Top 15 importances — PTS:


usage_share_roll5    0.482456
USG%                 0.090536
TS_game_roll10       0.045736
MIN_roll5            0.043958
TS_game_roll5        0.043564
MIN_roll10           0.041790
PTS_roll10           0.040254
PTS_roll5            0.033015
ORtg_g_roll5         0.031883
TS%                  0.025988
PER                  0.023671
Pace_g_roll5         0.021363
VORP                 0.021111
BPM                  0.020008
WS/48                0.015846
dtype: float64


Top 15 importances — REB:


REB_roll5            0.407061
REB_roll10           0.067645
TS_game_roll5        0.060479
TS_game_roll10       0.060150
usage_share_roll5    0.057093
MIN_roll10           0.052025
MIN_roll5            0.051689
PER                  0.044327
WS/48                0.033217
TS%                  0.029726
USG%                 0.025026
ORtg_g_roll5         0.024518
VORP                 0.022242
Pace_g_roll5         0.022217
BPM                  0.021760
dtype: float64


Top 15 importances — AST:


AST_roll5            0.452974
AST_roll10           0.070789
TS_game_roll10       0.054000
TS_game_roll5        0.052377
usage_share_roll5    0.052266
MIN_roll10           0.045795
MIN_roll5            0.044593
USG%                 0.034208
VORP                 0.031465
ORtg_g_roll5         0.031287
TS%                  0.026002
PER                  0.021997
Pace_g_roll5         0.021815
WS/48                0.021601
BPM                  0.018444
dtype: float64

## team-level predictions

In [124]:
# Team game table
team_games = features_all.groupby(["GAME_DATE","TEAM_ABBREVIATION"], as_index=False)\
    .agg(
        team_pts=("PTS","sum"),
        team_pts_next=("PTS_next","sum"),
        or5=("ORtg_g_roll5","mean"),
        dr5=("DRtg_g_roll5","mean"),
        pace5=("Pace_g_roll5","mean"),
    )

# Join opponent features (same date)
opp = team_games.rename(columns={
    "TEAM_ABBREVIATION":"OPPONENT_ABBREVIATION",
    "team_pts":"opp_pts",
    "team_pts_next":"opp_pts_next",
    "or5":"opp_or5","dr5":"opp_dr5","pace5":"opp_pace5"
})
team_matchups = team_games.merge(opp, on=["GAME_DATE"], how="inner")

# Simple features for team total prediction
team_feature_cols = ["or5","dr5","pace5","opp_or5","opp_dr5","opp_pace5"]
tm = team_matchups.dropna(subset=team_feature_cols + ["team_pts_next"]).copy()

from sklearn.linear_model import Ridge
X_tm = tm[team_feature_cols]
y_tm = tm["team_pts_next"]
ridge = Ridge(alpha=5.0).fit(X_tm, y_tm)
print("Team PTS baseline R^2:", ridge.score(X_tm, y_tm))


Team PTS baseline R^2: 0.15181475578664994


## Lineups

In [125]:
# pip install selenium webdriver-manager bs4 pandas lxml

import os, re, time, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------------- helpers ----------------

def _clean_list(xs):
    return [re.sub(r"\s+\(.*?\)\s*$", "", x) for x in xs]

def _try_click_consent(driver, timeout=6):
    XPATHS = [
        "//button[contains(.,'Accept')]",
        "//button[contains(.,'I Agree')]",
        "//button[contains(.,'Agree')]",
        "//button[contains(.,'Αποδοχή')]",
        "//button[contains(.,'Συμφωνώ')]",
    ]
    end = time.time() + timeout
    for xp in XPATHS:
        try:
            btn = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, xp)))
            btn.click()
            return True
        except Exception:
            if time.time() > end: break
    return False

def _progress_scroll(driver, steps=10, pause=0.8):
    h = driver.execute_script("return document.body.scrollHeight || document.documentElement.scrollHeight;")
    for i in range(1, steps + 1):
        y = int(h * i / steps)
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(pause)

def _extract_team(side):
    team_el = side.select_one(".lineup__abbr, .lineup__team-name, .lineup__name")
    if team_el:
        return team_el.get_text(strip=True)
    logo = side.select_one("img[alt]")
    return (logo.get("alt") or "").strip() if logo else ""

def _extract_status(side):
    status_el = side.select_one(".lineup__status")
    txt = (status_el.get_text(" ", strip=True) if status_el else "").upper()
    if "CONFIRM" in txt:  return "CONFIRMED"
    if "EXPECT" in txt or "PROBABLE" in txt: return "EXPECTED"
    return "UNKNOWN"

def _extract_starters(side):
    # Try several variants for starters content
    containers = side.select(".lineup__list--starters, .lineup__list, .lineup__players")
    if not containers:
        containers = [side]

    names = []
    for blk in containers:
        for a in blk.select("a.lineup__player-link, .lineup__player a"):
            t = a.get_text(" ", strip=True)
            if t: names.append(t)
        if not names:
            for row in blk.select(".lineup__player"):
                t = row.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)
        if not names:
            for li in blk.select("li"):
                t = li.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)

    if not names:
        txt = side.get_text("\n", strip=True)
        names = re.findall(r"(?:^|\n)(?:PG|SG|SF|PF|C)\s+[^\n]+", txt)

    return _clean_list(names)[:5]

# ---------------- main ----------------

def fetch_rotowire_lineups_selenium(date: str | None = None,
                                    wait_sec: float = 14.0,
                                    headless: bool = False) -> pd.DataFrame:
    """
    Render Rotowire lineups & parse BOTH sides per game (global side selectors).
    Returns:
      game_time, team, side (AWAY/HOME), lineup_status, starters,
      starter_1..starter_5, lineup_confirmed (0/1)
    """
    base = "https://www.rotowire.com/basketball/nba-lineups.php"
    url = base if not date else f"{base}?date={date}"

    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1400,1000")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--lang=en-US,en;q=0.9")
    opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.get(url)

    _try_click_consent(driver, timeout=6)
    time.sleep(1.2)
    try:
        WebDriverWait(driver, int(wait_sec)).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".lineup, .lineup.is-nba"))
        )
    except Exception:
        pass

    _progress_scroll(driver, steps=10, pause=0.8)
    time.sleep(1.0)

    # quick diagnostics
    blocks = driver.find_elements(By.CSS_SELECTOR, ".lineup.is-nba, .lineup")
    players = driver.find_elements(By.CSS_SELECTOR, ".lineup__player, a.lineup__player-link")
    print(f"diagnostics: lineup blocks={len(blocks)}, player nodes={len(players)}")

    html = driver.page_source
    os.makedirs("_rotowire_debug", exist_ok=True)
    with open("_rotowire_debug/last_lineups.html", "w", encoding="utf-8") as f:
        f.write(html)
    try:
        driver.save_screenshot("_rotowire_debug/last_lineups.png")
    except Exception:
        pass
    driver.quit()

    # -------- parse globally by side classes ----------
    soup = BeautifulSoup(html, "lxml")

    # game time map: find each game container time
    game_time_map = {}
    for gi, g in enumerate(soup.select(".lineup__main, .lineup.is-nba, .lineup")):
        t = g.select_one(".lineup__time, .game-time")
        game_time_map[id(g)] = t.get_text(strip=True) if t else ""

    # Select **visit/away** & **home** side boxes explicitly
    visit_sel = (
        '[class*="lineup__box"][class*="is-visit"], '
        '[class*="lineup__team"][class*="is-visit"], '
        '[class*="lineup__side"][class*="is-visit"], '
        '[class*="visit"]'
    )
    home_sel = (
        '[class*="lineup__box"][class*="is-home"], '
        '[class*="lineup__team"][class*="is-home"], '
        '[class*="lineup__side"][class*="is-home"], '
        '[class*="home"]'
    )

    visit_boxes = soup.select(visit_sel)
    home_boxes  = soup.select(home_sel)

    rows = []

    def add_rows(boxes, side_label):
        for box in boxes:
            # nearest parent game container for time
            parent = box.find_parent(lambda tag: tag.has_attr("class") and any(
                c in {"lineup__main","lineup","lineup is-nba"} for c in tag.get("class", [])
            ))
            game_time = game_time_map.get(id(parent), "") if parent else ""
            team = _extract_team(box)
            starters = _extract_starters(box)
            status = _extract_status(box)
            if starters or team:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side_label,
                    "lineup_status": status,
                    "starters": starters,
                    "starter_1": starters[0] if len(starters)>0 else None,
                    "starter_2": starters[1] if len(starters)>1 else None,
                    "starter_3": starters[2] if len(starters)>2 else None,
                    "starter_4": starters[3] if len(starters)>3 else None,
                    "starter_5": starters[4] if len(starters)>4 else None,
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    add_rows(visit_boxes, "AWAY")
    add_rows(home_boxes,  "HOME")

    df = pd.DataFrame(rows)

    if not df.empty:
        df = df.drop_duplicates(
            subset=["game_time","team","side","starter_1","starter_2","starter_3","starter_4","starter_5"]
        )
        all_na = df[["starter_1","starter_2","starter_3","starter_4","starter_5"]].isna().all(axis=1)
        df = df[~all_na].reset_index(drop=True)
    else:
        print("⚠️ Parsed zero rows. Check _rotowire_debug/last_lineups.html & .png")

    return df




In [126]:
# ---------- run it ----------
df_lineups = fetch_rotowire_lineups_selenium(wait_sec=14.0, headless=False)
print("✅ Shape:", df_lineups.shape)
print(df_lineups.sort_values(["game_time","side"]).head(12).to_string(index=False))


diagnostics: lineup blocks=11, player nodes=136
✅ Shape: (16, 11)
game_time team side lineup_status                                                                           starters             starter_1        starter_2     starter_3     starter_4        starter_5  lineup_confirmed
               AWAY     CONFIRMED                  [J. Fears, Trey Murphy, Herbert Jones, Z. Williamson, Yves Missi]              J. Fears      Trey Murphy Herbert Jones Z. Williamson       Yves Missi                 1
               AWAY      EXPECTED     [N. Alexander-Walker, Dyson Daniels, Z. Risacher, Jalen Johnson, K. Porzingis]   N. Alexander-Walker    Dyson Daniels   Z. Risacher Jalen Johnson     K. Porzingis                 0
               AWAY      EXPECTED                [Tyrese Maxey, VJ Edgecombe, Kelly Oubre, Jabari Walker, Adem Bona]          Tyrese Maxey     VJ Edgecombe   Kelly Oubre Jabari Walker        Adem Bona                 0
               AWAY      EXPECTED         [Cam Spencer, K.

## Selenium rotowire search

In [127]:
# pip install bs4 lxml pandas
import re, os, pandas as pd
from bs4 import BeautifulSoup

def _txt(x):
    return re.sub(r"\s+", " ", x.get_text(" ", strip=True)) if x else ""

def _clean_player(n):
    if not n: return n
    n = re.sub(r"\s+\(.*?\)\s*$", "", n).strip()
    n = re.sub(r"^(PG|SG|SF|PF|C)\s+", "", n, flags=re.I)
    return n

def _get_mnp_from_ul(ul):
    """Extract 'May Not Play' entries from a team UL."""
    mnp = []
    # Strategy 1: find the title li inside this UL, then collect following player lis until next title
    title = ul.find("li", class_=lambda c: c and "lineup__title" in c and re.search(r"may\s+not\s+play", _txt(ul.find("li", class_=c)) if ul.find("li", class_=c) else "", re.I))
    if title:
        for li in title.find_all_next("li"):
            # stop if next section title
            if "lineup__title" in (li.get("class") or []):
                break
            if "lineup__player" in (li.get("class") or []):
                a = li.select_one("a")
                tag = li.select_one(".lineup__inj")
                nm = _txt(a) if a else ""
                if nm:
                    mnp.append(f"{nm} ({_txt(tag)})" if tag else nm)
        # normalize
        return [_clean_player(x) for x in mnp if x and x.lower() != "none"]

    # Strategy 2: common MNP containers inside UL
    for li in ul.select(".lineup__notplay li, .lineup__status--out, .lineup__inj-list li"):
        nm = _txt(li)
        if nm: mnp.append(_clean_player(nm))
    return [x for x in mnp if x and x.lower() != "none"]

def _extract_starters_from_ul(ul):
    """Try multiple ways to get five starters out of a team UL."""
    names = []
    # Most reliable: 100% rows
    for li in ul.select("li.lineup__player.is-pct-play-100 a"):
        nm = _txt(li)
        if nm: names.append(nm)
    # Fallback: any lineup__player anchors in first list group
    if len(names) < 5:
        for li in ul.select("li.lineup__player a"):
            nm = _txt(li)
            if nm: names.append(nm)
            if len(names) >= 5: break
    # Final cleanup + trim
    names = [_clean_player(n) for n in names]
    return names[:5]

def _lineup_status(ul):
    st = _txt(ul.select_one(".lineup__status"))
    stU = st.upper()
    if "CONFIRM" in stU: return "CONFIRMED"
    if "EXPECT" in stU or "PROBABLE" in stU: return "EXPECTED"
    return "UNKNOWN"

def parse_rotowire_lineups_flexible(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        html = f.read()
    soup = BeautifulSoup(html, "lxml")

    # --- Diagnostics to understand the DOM you have ---
    diag = {
        "lineup__teams": len(soup.select("div.lineup__teams")),
        "ul.lineup__list": len(soup.select("ul.lineup__list")),
        "ul.is-visit": len(soup.select("ul.lineup__list.is-visit")),
        "ul.is-home": len(soup.select("ul.lineup__list.is-home")),
        "see-proj-minutes buttons": len(soup.select("button.see-proj-minutes")),
        "header abbr": len(soup.select(".lineup__hdr .lineup__abbr")),
        "header team": len(soup.select(".lineup__hdr .lineup__team")),
        "player anchors": len(soup.select("a.lineup__player-link, .lineup__player a")),
        "MNP titles": len(soup.find_all(string=re.compile(r"^\s*may\s+not\s+play\s*$", re.I))),
    }
    print("DOM diagnostics:", diag)

    rows = []

    # ========== STRATEGY A: by matchup blocks ==========
    for teams_div in soup.select("div.lineup__teams"):
        # game time near this block (looks upward for a sibling header)
        time_el = teams_div.find_previous("div", class_="lineup__time")
        game_time = _txt(time_el)

        # find both team ULs inside this matchup
        uls = teams_div.select("ul.lineup__list")
        if len(uls) < 1:
            continue

        # Try to pair AWAY then HOME by class flags; else preserve order
        away_ul = None
        home_ul = None
        for ul in uls:
            classes = " ".join(ul.get("class", [])).lower()
            if "is-visit" in classes or "visit" in classes or "away" in classes:
                away_ul = ul
            if "is-home" in classes or "home" in classes:
                home_ul = home_ul or ul  # keep the first

        if away_ul is None and home_ul is None and len(uls) >= 2:
            away_ul, home_ul = uls[0], uls[1]
        elif away_ul is None and len(uls) >= 1:
            away_ul = uls[0]
        elif home_ul is None and len(uls) >= 2:
            # pick the other UL as home
            home_ul = next((u for u in uls if u is not away_ul), None)

        pairs = [("AWAY", away_ul), ("HOME", home_ul)]
        # Extract team code (prefer button data-team; else header abbrs in the same matchup)
        header_abbrs = [ _txt(el) for el in teams_div.select(".lineup__abbr") if _txt(el) ]
        # If header not inside teams_div, try its parent block
        if not header_abbrs:
            parent_main = teams_div.find_parent(["div","section"])
            if parent_main:
                header_abbrs = [ _txt(el) for el in parent_main.select(".lineup__abbr") if _txt(el) ]

        for idx, (side, ul) in enumerate(pairs):
            if not ul: continue
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            if not team and header_abbrs and idx < len(header_abbrs):
                team = header_abbrs[idx].upper()

            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            # Only add if we have at least a team or any player info
            if team or starters or mnp:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    # ========== STRATEGY B: fall back to any lineup ULs globally ==========
    if not rows:
        print("Fallback B: scanning all ul.lineup__list globally...")
        for ul in soup.select("ul.lineup__list"):
            # Guess side by class or position among siblings
            side = "AWAY" if "is-visit" in (ul.get("class") or []) else ("HOME" if "is-home" in (ul.get("class") or []) else None)
            # Team from button
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            if side and (team or starters or mnp):
                rows.append({
                    "game_time": "",  # unknown at this scope
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    # ========== STRATEGY C: header-driven pairing (very defensive) ==========
    if not rows:
        print("Fallback C: pairing by header labels and nearest lists...")
        for block in soup.select(".lineup, .lineup__main"):
            hdr = block.select(".lineup__hdr .lineup__abbr, .lineup__hdr .lineup__team")
            labels = [ _txt(x) for x in hdr if _txt(x) ]
            if len(labels) < 2:
                continue
            away_label, home_label = labels[:2]
            lists = block.select("ul.lineup__list")
            if len(lists) < 2:
                continue
            for side, lab, ul in [("AWAY", away_label, lists[0]), ("HOME", home_label, lists[1])]:
                starters = _extract_starters_from_ul(ul)
                mnp = _get_mnp_from_ul(ul)
                status = _lineup_status(ul)
                rows.append({
                    "game_time": _txt(block.select_one(".lineup__time, .game-time")),
                    "team": lab.upper(),
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    df = pd.DataFrame(rows)
    # Expand starters to columns for easier merging
    for i in range(5):
        col = f"starter_{i+1}"
        df[col] = df["starters"].apply(lambda xs: xs[i] if isinstance(xs, list) and len(xs) > i else None)

    print(f"→ Parsed rows: {len(df)}")
    return df

# ---- RUN IT (point to your saved file) ----
HTML_PATH = "_rotowire_debug/last_lineups.html"  # change if needed
if not os.path.exists(HTML_PATH):
    # if you uploaded as 'last_lineups.html' in current directory
    if os.path.exists("last_lineups.html"):
        HTML_PATH = "last_lineups.html"

df_lineups = parse_rotowire_lineups_flexible(HTML_PATH)

# Safe display
if df_lineups.empty:
    print("\n⚠️ Still empty. Please share the values printed in 'DOM diagnostics' (above).")
else:
    cols = ["game_time","team","side","lineup_status","may_not_play_count",
            "starter_1","starter_2","starter_3","starter_4","starter_5"]
    print("\n✅ Preview:")
    print(df_lineups[cols].sort_values(["game_time","side","team"], na_position="last").to_string(index=False))


DOM diagnostics: {'lineup__teams': 8, 'ul.lineup__list': 16, 'ul.is-visit': 8, 'ul.is-home': 8, 'see-proj-minutes buttons': 16, 'header abbr': 0, 'header team': 0, 'player anchors': 136, 'MNP titles': 16}
Fallback B: scanning all ul.lineup__list globally...
→ Parsed rows: 16

✅ Preview:
game_time team side lineup_status  may_not_play_count             starter_1        starter_2     starter_3     starter_4        starter_5
           ATL AWAY      EXPECTED                   7   N. Alexander-Walker    Dyson Daniels   Z. Risacher Jalen Johnson     K. Porzingis
           CHI AWAY      EXPECTED                   9             Tre Jones      Josh Giddey   Isaac Okoro Matas Buzelis       N. Vucevic
           MEM AWAY      EXPECTED                  10           Cam Spencer K. Caldwell-Pope  Jaylen Wells Jaren Jackson     Jock Landale
           MIA AWAY      EXPECTED                   9           D. Mitchell       A. Wiggins   Bam Adebayo   Kel'el Ware      D. Mitchell
           NOP AWAY   

In [128]:
# pip install bs4 lxml pandas
import os, re, pandas as pd
from bs4 import BeautifulSoup

HTML_PATH = "_rotowire_debug/last_lineups.html" if os.path.exists("_rotowire_debug/last_lineups.html") else "last_lineups.html"

LIKELIHOOD_MAP = {
    "is-pct-play-100": 100, "is-pct-play-90": 90, "is-pct-play-75": 75,
    "is-pct-play-60": 60, "is-pct-play-50": 50, "is-pct-play-40": 40,
    "is-pct-play-25": 25, "is-pct-play-10": 10, "is-pct-play-0": 0
}

def _txt(node): return re.sub(r"\s+", " ", node.get_text(" ", strip=True)) if node else ""
def _likelihood(classes): 
    for c in classes: 
        if c in LIKELIHOOD_MAP: 
            return LIKELIHOOD_MAP[c]
    return None

def parse_rotowire_mnp_final(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        soup = BeautifulSoup(f.read(), "lxml")

    rows = []
    games = soup.select("div.lineup.is-nba[data-lnum]")
    print(f"Found {len(games)} games in HTML.")

    for game in games:
        game_time = _txt(game.select_one(".lineup__time"))
        team_blocks = game.select(".lineup__team")
        teams = []
        for tb in team_blocks:
            abbr = _txt(tb.select_one(".lineup__abbr"))
            side = "AWAY" if "is-visit" in tb.get("class", []) else "HOME" if "is-home" in tb.get("class", []) else None
            teams.append((abbr, side))

        ul_lists = game.select("ul.lineup__list")
        for idx, ul in enumerate(ul_lists):
            if idx >= len(teams):  # mismatch safety
                continue
            team, side = teams[idx]
            mnp_title = ul.find("li", class_="lineup__title", string=lambda s: s and "MAY NOT PLAY" in s.upper())
            if not mnp_title:
                continue

            for li in mnp_title.find_next_siblings("li"):
                classes = li.get("class") or []
                if "lineup__title" in classes:
                    break
                if "lineup__player" not in classes:
                    continue

                pos = _txt(li.select_one(".lineup__pos"))
                a = li.select_one("a")
                player = _txt(a)
                if not player:
                    continue

                status = _txt(li.select_one(".lineup__inj"))
                title_text = (li.get("title") or "").strip()
                likelihood_pct = _likelihood(classes)

                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "position": pos,
                    "player": player,
                    "status": status,
                    "title_text": title_text,
                    "likelihood_pct": likelihood_pct
                })

    df = pd.DataFrame(rows)
    if df.empty:
        print("⚠️ No 'May Not Play' players found. Check if Rotowire changed markup.")
    else:
        df = df.sort_values(["game_time","side","team","player"]).reset_index(drop=True)
        print(f"✅ Parsed {len(df)} 'May Not Play' players across {df['team'].nunique()} teams.")
    return df


# ---- RUN ----
mnp_df = parse_rotowire_mnp_final(HTML_PATH)
if not mnp_df.empty:
    print(mnp_df.head(30).to_string(index=False))
    mnp_df.to_csv("may_not_play_players.csv", index=False)
    print("\nSaved: may_not_play_players.csv")


Found 8 games in HTML.
✅ Parsed 56 'May Not Play' players across 16 teams.
 game_time team side position       player status            title_text  likelihood_pct
3:30 PM ET  NOP AWAY        G    D. Murray    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        C  C. Holmgren    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        F  J. Williams    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        F  K. Williams    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        F      L. Dort    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        G     N. Topic    Out Very Unlikely To Play               0
3:30 PM ET  OKC HOME        F     O. Dieng   Ques   Very Likely To Play             100
3:30 PM ET  OKC HOME        C    T. Sorber    OFS   Very Likely To Play               0
6:00 PM ET  ATL AWAY        G   K. Wallace    Out Very Unlikely To Play               0
6:00 PM ET  ATL AWAY        G   Trae Young   

## Cell 15: odds math + Excel export

In [129]:
import pandas as pd
import numpy as np
from datetime import datetime

def american_to_prob(odds):
    if pd.isna(odds): return np.nan
    o = float(odds)
    return 100.0/(o+100.0) if o>0 else (-o)/(-o+100.0)

def devig_pair(p_over, p_under):
    if pd.isna(p_over) or pd.isna(p_under): return (np.nan, np.nan)
    s = p_over + p_under
    if s <= 0: return (np.nan, np.nan)
    return (p_over/s, p_under/s)

def kelly_fraction(p, american_odds, cap=0.25):
    if pd.isna(p) or pd.isna(american_odds): return 0.0
    o = float(american_odds)
    b = o/100.0 if o>0 else 100.0/(-o)
    f = (p*(b+1)-1)/b
    return float(max(0.0, min(f, cap)))

def ev_flat_over(p, american_odds):
    if pd.isna(p) or pd.isna(american_odds): return np.nan
    o = float(american_odds)
    win = o/100.0 if o>0 else 100.0/(-o)
    lose = 1.0
    return p*win - (1-p)*lose

# Normal CDF helper (if SciPy available) to turn mean/sd into p_over
try:
    from scipy.stats import norm
    def p_over_from_normal(mu, sd, line):
        if pd.isna(mu) or pd.isna(sd) or pd.isna(line) or sd <= 0: return np.nan
        return 1.0 - norm.cdf((line - mu)/sd)
except Exception:
    def p_over_from_normal(mu, sd, line): return np.nan

def build_value_bets_excel(
    df_projections, df_odds, outfile_path=None,
    join_keys=("player","team","opponent","market","line","book","game_date"),
    cap_kelly=0.25
):
    def _norm(x): return None if pd.isna(x) else str(x).strip()
    proj, odds = df_projections.copy(), df_odds.copy()
    for k in join_keys:
        if k in proj: proj[k] = proj[k].map(_norm)
        if k in odds: odds[k] = odds[k].map(_norm)

    merged = proj.merge(odds, on=list(join_keys), how="inner", suffixes=("", "_odds"))

    if "p_over_model" not in merged.columns or merged["p_over_model"].isna().all():
        merged["p_over_model"] = merged.apply(
            lambda r: p_over_from_normal(r.get("projection_mean"), r.get("projection_sd"), r.get("line")), axis=1
        )

    merged["p_over_imp"]  = merged["over_odds"].map(american_to_prob)
    merged["p_under_imp"] = merged["under_odds"].map(american_to_prob)
    merged[["p_over_fair","p_under_fair"]] = merged.apply(
        lambda r: pd.Series(devig_pair(r["p_over_imp"], r["p_under_imp"])), axis=1
    )

    merged["edge_over"]       = merged["p_over_model"] - merged["p_over_fair"]
    merged["kelly_frac_over"] = merged.apply(lambda r: kelly_fraction(r["p_over_model"], r["over_odds"], cap=cap_kelly), axis=1)
    merged["EV_over_1u"]      = merged.apply(lambda r: ev_flat_over(r["p_over_model"], r["over_odds"]), axis=1)
    merged["asof_date"]       = merged.get("asof_date") if "asof_date" in merged else datetime.utcnow().strftime("%Y-%m-%d")

    preferred = [
        "asof_date","game_date","book","player","team","opponent","market","line","lineup_status",
        "over_odds","under_odds","p_over_imp","p_under_imp","p_over_fair","p_under_fair","p_over_model",
        "edge_over","kelly_frac_over","EV_over_1u",
        "projected_minutes","projection_mean","projection_sd","start_prob",
        "opponent_allowance_idx","team_orating","opp_drating",
    ]
    cols = [c for c in preferred if c in merged.columns] + [c for c in merged.columns if c not in preferred]
    bets = merged[cols].sort_values(["edge_over","EV_over_1u"], ascending=False).reset_index(drop=True)

    summary = pd.DataFrame({
        "n_bets":[len(bets)],
        "avg_edge_pp":[bets["edge_over"].mean()*100.0 if len(bets) else np.nan],
        "avg_kelly_pct":[bets["kelly_frac_over"].mean()*100.0 if len(bets) else np.nan],
        "avg_ev_1u":[bets["EV_over_1u"].mean() if len(bets) else np.nan],
    })
    by_market = bets.groupby("market", dropna=False).agg(
        n=("player","count"),
        avg_edge_pp=("edge_over", lambda x: 100.0*x.mean()),
        avg_kelly_pct=("kelly_frac_over", lambda x: 100.0*x.mean()),
        avg_ev_1u=("EV_over_1u","mean")
    ).reset_index()
    by_book = bets.groupby("book", dropna=False).agg(
        n=("player","count"),
        avg_edge_pp=("edge_over", lambda x: 100.0*x.mean()),
        avg_ev_1u=("EV_over_1u","mean")
    ).reset_index()

    if outfile_path is None:
        outfile_path = f"nba_value_bets_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.xlsx"
    with pd.ExcelWriter(outfile_path, engine="openpyxl") as w:
        bets.to_excel(w, sheet_name="Bets", index=False)
        summary.to_excel(w, sheet_name="Summary", index=False, startrow=0)
        by_market.to_excel(w, sheet_name="Summary", index=False, startrow=5)
        by_book.to_excel(w, sheet_name="Summary", index=False, startrow=5+len(by_market)+3)

        dd = pd.DataFrame([
            ("asof_date","UTC run date"), ("game_date","Game date"),
            ("player","Player"), ("team","Team abbr"), ("opponent","Opponent abbr"),
            ("market","PTS/REB/AST/3PM/PRA etc."), ("line","Book line"), ("book","Sportsbook id"),
            ("lineup_status","EXPECTED/CONFIRMED/UNKNOWN"),
            ("over_odds","American odds Over"), ("under_odds","American odds Under"),
            ("p_over_imp","Implied prob Over (pre-vig)"), ("p_under_imp","Implied prob Under (pre-vig)"),
            ("p_over_fair","De-vigged prob Over"), ("p_under_fair","De-vigged prob Under"),
            ("p_over_model","Model prob Over"), ("edge_over","p_model − p_fair"),
            ("kelly_frac_over","Kelly fraction (cap)"), ("EV_over_1u","EV if staking 1u"),
            ("projected_minutes","Projected minutes"), ("projection_mean","Projected mean"),
            ("projection_sd","Projected stdev"), ("start_prob","Start probability"),
            ("opponent_allowance_idx","Opponent allowance index"),
            ("team_orating","Team ORtg"), ("opp_drating","Opponent DRtg"),
        ], columns=["column","description"])
        dd.to_excel(w, sheet_name="Data_Dictionary", index=False)

    return bets, outfile_path


In [130]:
# === 16: raw wide odds + resilient numeric parsing ===
import re, json, pandas as pd
from datetime import datetime

def _first_numeric_float(x):
    """Return the first decimal number in x (e.g., '23.5, 24.5' -> 23.5)."""
    if x is None: return None
    s = str(x)
    m = re.search(r"[-+]?\d+(?:\.\d+)?", s)
    return float(m.group()) if m else None

def _first_numeric_int(x):
    """Return the first integer in x (e.g., '+110, +105' -> 110)."""
    if x is None: return None
    s = str(x)
    m = re.search(r"[-+]?\d+", s)
    return int(m.group()) if m else None

# override the helpers used by 16d converter (if defined)
def _to_float_or_none(x):  # noqa: F811
    return _first_numeric_float(x)

def _to_int_or_none(x):    # noqa: F811
    return _first_numeric_int(x)

def get_player_props_odds_wide_raw(self, book="mgm"):
    """
    Returns the raw 'wide' odds table from Rotowire (no grouping, no aggregation).
    Contains columns like mgm_pts, mgm_ptsOver, mgm_ptsUnder, etc.
    """
    url = f"https://www.rotowire.com/betting/nba/player-props.php?book={book}"
    r = self.session.get(url, headers=self.headers, timeout=30)
    r.raise_for_status()
    matches = re.findall(r'data:\s*(\[\{.*?\}\])', r.text, flags=re.DOTALL)
    frames = []
    for blob in matches:
        try:
            frames.append(pd.DataFrame(json.loads(blob)))
        except Exception:
            pass
    if not frames:
        return pd.DataFrame()
    # concat all blocks without grouping to preserve raw book columns
    wide_raw = pd.concat(frames, ignore_index=True)
    return wide_raw

# attach to your scraper class
NBAOddsAndLineupsScraper.get_player_props_odds_wide_raw = get_player_props_odds_wide_raw


In [131]:
# === Cell 16: projections for PTS/REB/AST using your trained RF models ===
import pandas as pd
import numpy as np
from datetime import datetime

# Safety checks
if "models" not in globals() or not models:
    raise RuntimeError("No trained models found. Run Cell 7 first to populate `models` and `feature_cols_by_stat`.")

# We'll project for these markets
MARKETS = ["PTS", "REB", "AST"]

# Latest row per player as basis for "next game"
latest = features_all.sort_values(["PLAYER_NAME","GAME_DATE"]).groupby("PLAYER_NAME").tail(1).copy()

# Helper: per-stat stdev from last N actual games
def _player_sd_map(stat: str, n=10):
    def _sd(g):
        s = g[stat].tail(n)
        if s.notna().sum() >= 4:
            return float(s.std(ddof=1))
        return float(features_all[stat].std(ddof=1))
    return features_all.groupby("PLAYER_NAME").apply(_sd)

# Normalize export keys common to all markets
base_cols = {
    "PLAYER_NAME": "player",
    "TEAM_ABBREVIATION": "team",
    "OPPONENT_ABBREVIATION": "opponent",
}
base_out = latest.rename(columns=base_cols)[["player","team","opponent"]].copy()
base_out["game_date"] = datetime.utcnow().strftime("%Y-%m-%d")
base_out["projected_minutes"] = latest.get("MIN_roll5", pd.Series(index=latest.index)).fillna(30).clip(lower=10, upper=40).values
base_out["start_prob"] = 0.90
base_out["lineup_status"] = "EXPECTED"

# Build one projection frame per market
proj_frames = {}
for stat in MARKETS:
    if stat not in models:
        print(f"⚠️ Skipping {stat}: model not found in `models`.")
        continue
    feat_cols = feature_cols_by_stat.get(stat, [])
    if not feat_cols:
        print(f"⚠️ Skipping {stat}: no feature columns recorded in `feature_cols_by_stat`.")
        continue

    X_pred = latest[feat_cols].fillna(method="ffill").fillna(0)
    pred_mean = models[stat].predict(X_pred)

    # per-player SD
    sd_map = _player_sd_map(stat)
    pred_sd = latest["PLAYER_NAME"].map(sd_map)
    # conservative fallback SD = 15% of mean (min 1.0)
    sd_fallback = np.maximum(np.abs(pred_mean) * 0.15, 1.0)
    pred_sd = np.where(np.isnan(pred_sd), sd_fallback, pred_sd)

    dfp = base_out.copy()
    dfp["projection_mean"] = pred_mean
    dfp["projection_sd"] = pred_sd
    dfp["market"] = stat

    # Expose per-market frames
    proj_frames[stat] = dfp[["player","team","opponent","game_date","market",
                             "projection_mean","projection_sd","projected_minutes","start_prob","lineup_status"]].copy()

# Individual frames (kept for backward compatibility)
df_projections_pts = proj_frames.get("PTS", pd.DataFrame())
df_projections_reb = proj_frames.get("REB", pd.DataFrame())
df_projections_ast = proj_frames.get("AST", pd.DataFrame())

# Combined projections across markets
df_projections_all = pd.concat(list(proj_frames.values()), ignore_index=True) if proj_frames else pd.DataFrame()

print("Projection rows by market:",
      {k: len(v) for k, v in proj_frames.items()})


Projection rows by market: {'PTS': 694, 'REB': 694, 'AST': 694}


In [138]:
# === Cell 17: wide_raw → long (PTS/REB/AST) → join → export ===
from datetime import datetime
import re, unicodedata
import numpy as np
import pandas as pd

scraper = NBAOddsAndLineupsScraper()

# 1) Raw wide odds (no grouping)
wide_raw = scraper.get_player_props_odds_wide_raw(book="mgm")
if wide_raw.empty:
    raise RuntimeError("Raw wide odds are empty. The page structure may have changed or was blocked.")

print("Total rows in raw wide:", len(wide_raw))

# 2) Detect books present
books_seen = sorted({
    m.group(1) for c in wide_raw.columns
    if (m := re.match(r"^(draftkings|fanduel|caesars|betrivers|espnbet|hardrock|mgm)_(.+)$", c))
})
print("Books present in columns:", books_seen)

def _col_exists_nonnull(df, col):
    return (col in df.columns) and df[col].notna().any()

# Which books have each market today?
market_suffix = {"PTS":"pts","REB":"reb","AST":"ast"}
books_by_market = {
    m: [b for b in books_seen if _col_exists_nonnull(wide_raw, f"{b}_{market_suffix[m]}")]
    for m in ["PTS","REB","AST"]
}
print("Books with lines:", {m: v for m, v in books_by_market.items() if v})

# 3) Convert wide → long for markets that actually have any lines
target_markets = tuple([m for m, bs in books_by_market.items() if bs])
if not target_markets:
    raise RuntimeError("No books have non-null PTS/REB/AST lines today.")

odds_long = odds_wide_to_long_from_columns(
    wide_raw,
    books=tuple(sorted({b for bs in books_by_market.values() for b in bs})),
    markets=target_markets
)
if odds_long.empty:
    raise RuntimeError("odds_long is empty after conversion. Verify your `odds_wide_to_long_from_columns` mapping.")

# Normalize obvious numerics
def _num_float(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
    return float(m.group()) if m else np.nan

def _num_int(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+", str(x))
    return int(m.group()) if m else np.nan

odds_long["line"] = odds_long["line"].apply(_num_float)
odds_long["over_odds"] = odds_long["over_odds"].apply(_num_int)
odds_long["under_odds"] = odds_long["under_odds"].apply(_num_int)

print("odds_long rows:", len(odds_long))
print("odds_long columns:", odds_long.columns.tolist())
print(odds_long.head(5))

# 4) Prepare projections union (must be created earlier, e.g., Cell 16e)
if "df_projections_all" not in globals() or df_projections_all.empty:
    raise RuntimeError("df_projections_all not found or empty (run Cell 16 that builds PTS/REB/AST projections).")

# Light name normalizer
def _norm_player(name: str) -> str:
    if not isinstance(name, str): return ""
    s = unicodedata.normalize("NFKD", name)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[.\-`'’]", "", s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    return s

odds_long = odds_long.copy()
df_projections_all = df_projections_all.copy()

odds_long["player_key"] = odds_long["player"].map(_norm_player)
df_projections_all["player_key"] = df_projections_all["player"].map(_norm_player)

# Ensure projection SD exists per market (fallback to 15% of mean, min 1.0)
for m in ["PTS","REB","AST"]:
    mask = df_projections_all["market"].eq(m)
    if "projection_sd" not in df_projections_all.columns:
        df_projections_all["projection_sd"] = np.nan
    missing_sd = df_projections_all.loc[mask, "projection_sd"].isna() | (df_projections_all.loc[mask, "projection_sd"] <= 0)
    if missing_sd.any():
        df_projections_all.loc[mask, "projection_sd"] = (
            df_projections_all.loc[mask, "projection_mean"].abs() * 0.15
        ).clip(lower=1.0)

# 5) Split and join per market, then combine
joined_frames = []
for mkt in target_markets:
    odds_m = odds_long.loc[odds_long["market"].eq(mkt)].copy()
    proj_m = df_projections_all.loc[df_projections_all["market"].eq(mkt)].copy()
    if odds_m.empty or proj_m.empty:
        print(f"⚠️ Skipping {mkt}: odds or projections empty.")
        continue

    join_keys = ["player_key","market"]
    view_cols_odds = join_keys + ["player","team","opponent","line","book","game_date","over_odds","under_odds"]
    view_cols_odds = [c for c in view_cols_odds if c in odds_m.columns]

    view_cols_proj = join_keys + [
        "player","team","opponent","game_date","projection_mean","projection_sd",
        "projected_minutes","start_prob","lineup_status"
    ]
    view_cols_proj = [c for c in view_cols_proj if c in proj_m.columns]

    dfj = proj_m[view_cols_proj].merge(
        odds_m[view_cols_odds].rename(columns={"player":"player_odds","team":"team_odds","opponent":"opponent_odds","game_date":"game_date_odds"}),
        on=join_keys, how="inner", suffixes=("_proj","_odds")
    )

    if dfj.empty:
        print(f"⚠️ Join produced 0 rows for {mkt}. Check name variants.")
        continue

    # Resolve canonical columns
    def _pick_first(df_, names, default=np.nan):
        for n in names:
            if n in df_.columns:
                return df_[n]
        return default

    dfj = dfj.loc[:, ~dfj.columns.duplicated()].copy()
    dfj["player"]    = _pick_first(dfj, ["player_odds","player_proj","player"])
    dfj["team"]      = _pick_first(dfj, ["team_odds","team_proj","team"])
    dfj["opponent"]  = _pick_first(dfj, ["opponent_odds","opponent_proj","opponent"])
    dfj["game_date"] = _pick_first(dfj, ["game_date_odds","game_date_proj","game_date"])
    # line already numeric above, but if any slipped through:
    dfj["line"] = dfj["line"].apply(_num_float)

    # Compute model probability P(Over)
    if "p_over_from_normal" not in globals():
        from statistics import NormalDist
        def p_over_from_normal(mu, sd, line):
            if pd.isna(mu) or pd.isna(sd) or pd.isna(line) or float(sd) <= 0: return np.nan
            z = (float(line) - float(mu)) / float(sd)
            return 1.0 - NormalDist().cdf(z)

    dfj["p_over_model"] = dfj.apply(
        lambda r: p_over_from_normal(r.get("projection_mean"), r.get("projection_sd"), r.get("line")), axis=1
    )

    # Implied/fair probabilities + edge (so we can pick best book later)
    def implied_prob(american):
        o = _num_int(american)
        if pd.isna(o): return np.nan
        return (-o)/(-o+100.0) if o < 0 else 100.0/(o+100.0)

    dfj["p_over_imp"]  = dfj["over_odds"].apply(implied_prob)
    dfj["p_under_imp"] = dfj["under_odds"].apply(implied_prob)

    def devig_pair(p_over_imp, p_under_imp):
        if pd.isna(p_over_imp) or pd.isna(p_under_imp):
            return (np.nan, np.nan)
        s = p_over_imp + p_under_imp
        if s <= 0:
            return (np.nan, np.nan)
        return (p_over_imp/s, p_under_imp/s)

    fair = dfj.apply(
        lambda r: pd.Series(devig_pair(r["p_over_imp"], r["p_under_imp"]), index=["p_over_fair","p_under_fair"]),
        axis=1
    )
    dfj = pd.concat([dfj, fair], axis=1)

    dfj["edge_over"] = np.where(
        dfj["p_over_fair"].notna(),
        dfj["p_over_model"] - dfj["p_over_fair"],
        dfj["p_over_model"] - dfj["p_over_imp"]
    )

    # Keep the best book per player/market (highest edge)
    dfj = dfj.sort_values(["player","market","edge_over"], ascending=[True, True, False])
    dfj = dfj.drop_duplicates(subset=["player","market"], keep="first")

    joined_frames.append(
        dfj[[
            "player","team","opponent","market","line","book","game_date",
            "over_odds","under_odds","projection_mean","projection_sd",
            "projected_minutes","start_prob","lineup_status","p_over_model","edge_over"
        ]]
    )

# Combined joined frame for all markets
if not joined_frames:
    raise RuntimeError("No joined rows produced for any market.")
df_proj_join_all = pd.concat(joined_frames, ignore_index=True)

# 6) Prepare odds slice for Excel builder
df_odds_for_excel = df_proj_join_all[[
    "player","team","opponent","market","line","book","game_date","over_odds","under_odds"
]].copy()

# 7) Export full bets workbook
bets, excel_path = build_value_bets_excel(
    df_projections=df_proj_join_all,
    df_odds=df_odds_for_excel,
    outfile_path=f"nba_value_bets_{datetime.utcnow().strftime('%Y%m%d')}.xlsx",
    join_keys=("player","market")  # permissive merge
)

print(f"\nSaved value bets to: {excel_path}")
print(len(bets), "value bets found across", df_proj_join_all['market'].nunique(), "markets.")
display(bets.head(20))


Total rows in raw wide: 1661
Books present in columns: ['betrivers', 'caesars', 'draftkings', 'espnbet', 'fanduel', 'hardrock', 'mgm']
Books with lines: {'PTS': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm'], 'REB': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm'], 'AST': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm']}
odds_long rows: 1723
odds_long columns: ['asof_date', 'book', 'player', 'team', 'opponent', 'market', 'line', 'over_odds', 'under_odds', 'game_date']
    asof_date        book       player team opponent market  line  over_odds  \
0  2025-11-02   BETRIVERS  Luka Doncic  LAL      MIA    PTS  33.5       -106   
1  2025-11-02     CAESARS  Luka Doncic  LAL      MIA    PTS  33.5       -114   
2  2025-11-02  DRAFTKINGS  Luka Doncic  LAL      MIA    PTS  33.5       -109   
3  2025-11-02     FANDUEL  Luka Doncic  LAL      MIA    PTS  32.5       -122   
4  2025-11-02    HARDROCK  Luka Doncic  LAL      MIA    PTS  34.5

,asof_date,game_date,book,player,team,opponent,market,line,lineup_status,over_odds,...,projection_mean,projection_sd,start_prob,team_odds,opponent_odds,line_odds,book_odds,game_date_odds,over_odds_odds,under_odds_odds
0,2025-11-02,2025-11-02,DRAFTKINGS,Aaron Wiggins,OKC,NOP,AST,2.5,EXPECTED,140,...,6.986667,1.048000,0.9,OKC,NOP,2.5,DRAFTKINGS,2025-11-02,140,-183
1,2025-11-02,2025-11-02,FANDUEL,Jamal Shead,TOR,MEM,AST,4.5,EXPECTED,134,...,8.957333,1.343600,0.9,TOR,MEM,4.5,FANDUEL,2025-11-02,134,-172
2,2025-11-02,2025-11-02,MGM,Miles McBride,NYK,CHI,AST,2.5,EXPECTED,130,...,7.988333,1.198250,0.9,NYK,CHI,2.5,MGM,2025-11-02,130,-175
3,2025-11-02,2025-11-02,FANDUEL,Cam Thomas,BKN,PHI,AST,4.5,EXPECTED,132,...,9.638750,1.445812,0.9,BKN,PHI,4.5,FANDUEL,2025-11-02,132,-170
4,2025-11-02,2025-11-02,DRAFTKINGS,Patrick Williams,CHI,@NYK,REB,3.5,EXPECTED,132,...,5.968042,1.000000,0.9,CHI,@NYK,3.5,DRAFTKINGS,2025-11-02,132,-172
5,2025-11-02,2025-11-02,CAESARS,Isaiah Joe,OKC,NOP,REB,3.5,EXPECTED,125,...,5.964167,1.000000,0.9,OKC,NOP,3.5,CAESARS,2025-11-02,125,-185
6,2025-11-02,2025-11-02,FANDUEL,Josh Giddey,CHI,@NYK,REB,8.5,EXPECTED,116,...,15.633720,2.345058,0.9,CHI,@NYK,8.5,FANDUEL,2025-11-02,116,-148
7,2025-11-02,2025-11-02,CAESARS,Dyson Daniels,ATL,@CLE,REB,6.5,EXPECTED,115,...,9.875780,1.481367,0.9,ATL,@CLE,6.5,CAESARS,2025-11-02,115,-157
8,2025-11-02,2025-11-02,HARDROCK,Ayo Dosunmu,CHI,@NYK,REB,2.5,EXPECTED,110,...,5.002708,1.000000,0.9,CHI,@NYK,2.5,HARDROCK,2025-11-02,110,-155
9,2025-11-02,2025-11-02,DRAFTKINGS,Jaxson Hayes,LAL,MIA,PTS,4.5,EXPECTED,116,...,6.943810,1.041571,0.9,LAL,MIA,4.5,DRAFTKINGS,2025-11-02,116,-148


In [141]:
# === Cell: Build & export Top-100 Value Bets (PTS/REB/AST) — robust & lean ===
import numpy as np
import pandas as pd
from datetime import datetime
import re

# ---- 0) Source table ---------------------------------------------------------
if "df_proj_join_all" in globals() and isinstance(df_proj_join_all, pd.DataFrame) and not df_proj_join_all.empty:
    df = df_proj_join_all.copy()
elif "bets" in globals() and isinstance(bets, pd.DataFrame) and not bets.empty:
    df = bets.copy()
elif "df_proj_join" in globals() and isinstance(df_proj_join, pd.DataFrame) and not df_proj_join.empty:
    df = df_proj_join.copy()
else:
    raise RuntimeError("No joined dataset found (df_proj_join_all/bets/df_proj_join). Run the join cell first.")

# ---- 1) Canonicalize minimal fields -----------------------------------------
def _pick(df_, names):
    for n in names:
        if n in df_.columns:
            return df_[n]
    return pd.Series([np.nan]*len(df_))

df = df.copy()
df["player"]          = _pick(df, ["player","player_proj","player_odds"])
df["team"]            = _pick(df, ["team","team_proj","team_odds"])
df["opponent"]        = _pick(df, ["opponent","opponent_proj","opponent_odds"])
df["market"]          = _pick(df, ["market"])
df["line"]            = _pick(df, ["line","posted_line","book_line"])
df["book"]            = _pick(df, ["book"])
df["over_odds"]       = _pick(df, ["over_odds"])
df["under_odds"]      = _pick(df, ["under_odds"])
df["projection_mean"] = _pick(df, ["projection_mean","expected_line"])
df["projection_sd"]   = _pick(df, ["projection_sd"])

# ---- 2) Coerce numerics ------------------------------------------------------
def _first_float(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
    return float(m.group()) if m else np.nan

def _first_int(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+", str(x))
    return float(m.group()) if m else np.nan

for col in ["line","projection_mean","projection_sd"]:
    df[col] = df[col].apply(_first_float)

# ---- 3) Ensure p_over_model (with fallback SD) -------------------------------
if "p_over_model" not in df.columns or df["p_over_model"].isna().all():
    sd_missing = ("projection_sd" not in df.columns) or df["projection_sd"].fillna(0).eq(0).all()
    if sd_missing:
        df["projection_sd"] = (df["projection_mean"].abs() * 0.15).clip(lower=1.0)

    from statistics import NormalDist
    def p_over_from_normal(mean, sd, line):
        if pd.isna(mean) or pd.isna(sd) or pd.isna(line) or float(sd) <= 0:
            return np.nan
        z = (float(line) - float(mean)) / float(sd)
        return 1.0 - NormalDist().cdf(z)

    df["p_over_model"] = df.apply(
        lambda r: p_over_from_normal(r["projection_mean"], r["projection_sd"], r["line"]),
        axis=1
    )

# ---- 4) Implied / fair probs + edge (build if missing) -----------------------
def implied_prob_from_american(american):
    o = _first_int(american)
    if pd.isna(o): return np.nan
    return (-o)/(-o+100.0) if o < 0 else 100.0/(o+100.0)

if "p_over_imp" not in df.columns:
    df["p_over_imp"] = df["over_odds"].apply(implied_prob_from_american)
if "p_under_imp" not in df.columns:
    df["p_under_imp"] = df["under_odds"].apply(implied_prob_from_american)

if "p_over_fair" not in df.columns or "p_under_fair" not in df.columns:
    fair = df.apply(
        lambda r: pd.Series(
            (np.nan, np.nan) if (pd.isna(r["p_over_imp"]) or pd.isna(r["p_under_imp"])) else
            (r["p_over_imp"]/(r["p_over_imp"]+r["p_under_imp"]), r["p_under_imp"]/(r["p_over_imp"]+r["p_under_imp"]))
        , index=["p_over_fair","p_under_fair"]), axis=1)
    df = pd.concat([df, fair], axis=1)

if "edge_over" not in df.columns:
    df["edge_over"] = np.where(
        df["p_over_fair"].notna(),
        df["p_over_model"] - df["p_over_fair"],
        df["p_over_model"] - df["p_over_imp"]
    )

# ---- 5) Fallback ranking signal if probs weak: z-score of (expected - line) --
df["z_score"] = (df["projection_mean"] - df["line"]) / df["projection_sd"].replace(0, np.nan)
# If edge is NaN, use z_score scaled to pseudo-prob edge (~convert z to prob diff)
from statistics import NormalDist
nd = NormalDist()
df["edge_fallback"] = df["z_score"].map(lambda z: (nd.cdf(z) - 0.5)*2 if pd.notna(z) else np.nan)
df["edge_rank"] = np.where(df["edge_over"].notna(), df["edge_over"], df["edge_fallback"])

# ---- 6) Build lean, keep best book per player/market -------------------------
lean = df.loc[:, [
    "player","team","opponent","market","line","book","over_odds","under_odds",
    "projection_mean","p_over_model","edge_rank","edge_over"
]].copy()

# Drop rows missing essentials
lean = lean.dropna(subset=["player","market","line","projection_mean","book","over_odds"], how="any")

# Keep the best row per player/market by edge_rank
lean = lean.sort_values(["player","market","edge_rank"], ascending=[True, True, False])
lean = lean.drop_duplicates(subset=["player","market"], keep="first")

# ---- 7) Select Top-100: positives first, then fill with the next best --------
pos = lean[lean["edge_rank"] > 0].copy()
neg = lean[lean["edge_rank"] <= 0].copy().sort_values(["edge_rank","p_over_model"], ascending=[False, False])
top = pd.concat([pos, neg]).head(300).copy()

# ---- 8) Final formatting & export -------------------------------------------
top.rename(columns={"line":"posted_line","projection_mean":"expected_line"}, inplace=True)
for c in ["posted_line","expected_line"]:
    top[c] = pd.to_numeric(top[c], errors="coerce").round(1)

final_cols = ["player","team","opponent","market","posted_line","expected_line","book","over_odds","under_odds","edge_rank","p_over_model"]
top_out = top[final_cols].copy()

out_path = f"value_bets_top100_{datetime.utcnow().strftime('%Y%m%d')}.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as w:
    top_out.to_excel(w, sheet_name="Top100", index=False)

print(f"✅ Saved Top-100 value bets (PTS/REB/AST) to: {out_path}")
print(f"Positive-edge available: {len(pos)}; Exported: {len(top_out)}")
display(top_out.head(20))


✅ Saved Top-100 value bets (PTS/REB/AST) to: value_bets_top100_20251102.xlsx
Positive-edge available: 155; Exported: 300


,player,team,opponent,market,posted_line,expected_line,book,over_odds,under_odds,edge_rank,p_over_model
211,Aaron Wiggins,OKC,NOP,AST,2.5,7.0,DRAFTKINGS,140,-183,0.608132,0.999991
0,Aaron Wiggins,OKC,NOP,PTS,15.5,27.9,DRAFTKINGS,-109,-117,0.506768,0.998453
108,Aaron Wiggins,OKC,NOP,REB,4.5,7.0,HARDROCK,-130,-105,0.466619,0.991226
1,Adem Bona,PHI,@BKN,PTS,8.5,16.0,BETRIVERS,112,-106,0.520814,0.999082
109,Adem Bona,PHI,@BKN,REB,6.5,9.0,FANDUEL,108,-138,0.512892,0.966191
213,Alex Caruso,OKC,NOP,AST,2.5,3.0,HARDROCK,115,-165,0.272906,0.700498
3,Alex Caruso,OKC,NOP,PTS,8.5,18.9,MGM,105,-145,0.548059,0.999880
111,Alex Caruso,OKC,NOP,REB,3.5,5.0,FANDUEL,118,-150,0.499377,0.932653
214,Andrew Wiggins,MIA,@LAL,AST,2.5,3.0,MGM,110,-145,0.255007,0.700867
113,Andrew Wiggins,MIA,@LAL,REB,4.5,4.9,HARDROCK,-110,-130,0.192306,0.673295
